# Load Package

In [1]:
import math
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Load Dataset

In [6]:
train_df = pd.read_csv('D:/Kegiatan/Gammafest IPB/train.csv', sep=',')

# Preprocessing Data

## Handling Missing Values

In [7]:
train_dropna = train_df.copy()

### Mode Imputation

In [8]:
# Membuat fungsi yang dapat digunakan untuk melakukan imputasi dengan nilai modus
def mode_imputation(df, column):
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace=True)
    return df

In [9]:
train_dropna = mode_imputation(train_dropna, 'DC205')
train_dropna = mode_imputation(train_dropna, 'DC206')
train_dropna = mode_imputation(train_dropna, 'DC207')
train_dropna = mode_imputation(train_dropna, 'DC208')
train_dropna = mode_imputation(train_dropna, 'DC209')
train_dropna = mode_imputation(train_dropna, 'DC210')
train_dropna = mode_imputation(train_dropna, 'DC211')
train_dropna = mode_imputation(train_dropna, 'DC212')
train_dropna = mode_imputation(train_dropna, 'DC213')
train_dropna = mode_imputation(train_dropna, 'DC214')
train_dropna = mode_imputation(train_dropna, 'DC215')
train_dropna = mode_imputation(train_dropna, 'DC216')
train_dropna = mode_imputation(train_dropna, 'DC226')
train_dropna = mode_imputation(train_dropna, 'DC230a')
train_dropna = mode_imputation(train_dropna, 'DC230b')
train_dropna = mode_imputation(train_dropna, 'DC232')
train_dropna = mode_imputation(train_dropna, 'DC232b')
train_dropna = mode_imputation(train_dropna, 'DC235')
train_dropna = mode_imputation(train_dropna, 'DC237')
train_dropna = mode_imputation(train_dropna, 'DC237a')
train_dropna = mode_imputation(train_dropna, 'DC237b')
train_dropna = mode_imputation(train_dropna, 'DC237c')
train_dropna = mode_imputation(train_dropna, 'DC237d')
train_dropna = mode_imputation(train_dropna, 'DC237e')
train_dropna = mode_imputation(train_dropna, 'DC237f')
train_dropna = mode_imputation(train_dropna, 'DC241')
train_dropna = mode_imputation(train_dropna, 'DC242')
train_dropna = mode_imputation(train_dropna, 'DC244')
train_dropna = mode_imputation(train_dropna, 'DC246')
train_dropna = mode_imputation(train_dropna, 'DC252')
train_dropna = mode_imputation(train_dropna, 'DC270a')
train_dropna = mode_imputation(train_dropna, 'DC109')

### Median Imputation

In [10]:
# Membuat fungsi yang dapat digunakan untuk melakukan imputasi dengan nilai modus
def median_imputation(df, column):
    median_value = df[column].median()
    df[column].fillna(median_value, inplace=True)
    return df

In [11]:
train_dropna = median_imputation(train_dropna, 'DC142a')
train_dropna = median_imputation(train_dropna, 'DC220')

### Drop Missing Values

In [12]:
train_dropna = train_dropna.dropna().reset_index(drop=True)

### Data Transformation

In [13]:
# Melabeli setiap nilai pada fitur DC201
train_dropna['DC201'] = train_dropna['DC201'].replace('Layak Minum', 1)
train_dropna['DC201'] = train_dropna['DC201'].replace('Tidak Layak Minum', 0)

In [14]:
# Konversi tipe data pada fitur DC201
train_dropna['DC201'] = train_dropna['DC201'].astype('int64')

### Features Selection

In [15]:
train_selection = train_dropna.copy()

In [22]:
feature_columns = ['DC024', 'DC025', 'DC205', 'DC206', 'DC207', 'DC208', 
                   'DC209', 'DC210', 'DC211', 'DC212', 'DC213', 'DC214', 
                   'DC215', 'DC216','DC217', 'DC219','DC220', 'DC226', 'DC230a', 'DC230b', 
                   'DC232', 'DC232b', 'DC235', 'DC237', 'DC237a', 'DC237b', 
                   'DC237c', 'DC237d', 'DC237e', 'DC237f', 'DC241', 'DC242','DC244',
                   'DC246', 'DC252', 'DC270a', 'DC109','DC142a']

label_column = 'DC201'

In [31]:
# Menghitung nilai korelasi untuk setiap fitur dan nilai rataan dari seluruh korelasi
correlations = train_selection[feature_columns + [label_column]].corr()

corr_feature = []
jum = 0

for features in feature_columns:
    correlation_value = correlations.loc[features, label_column]
    
    corr_feature.append(correlation_value)
    
    if correlation_value < 0:
        correlation_value = abs(correlation_value)
    jum += correlation_value

In [32]:
# Menampilkan nilai korelasi untuk setiap fitur
print(corr_feature)

[-0.0243847167571317, -0.25638529431553364, -0.2708483199692181, 0.1617274277045489, 0.0620095046369823, 0.19210076329111822, 0.2149434975138483, 0.10855786896294307, 0.1327481596627701, 0.10454326178005408, 0.13041655115307327, 0.03186537875845745, 0.13750162673523347, 0.0828088837423915, 0.021948712731529794, -0.006992274776799389, -0.01992793914349794, -0.022886995341804082, -0.13990297523346987, 0.08504902250578414, 0.09865065060780812, -0.030825276481345174, -0.05662691834453649, -0.18150046011326088, -0.17699535927969604, -0.03053451684293955, -0.04463383891224175, -0.007319142704607651, 0.0060357903412041865, -0.09747038480424482, -0.09602370014515456, 0.011618138435613408, -0.17131158318038758, -0.15396102185713603, -0.020733536693352504, 0.23651402947060884, -0.2737224044660681, 0.07279861254873744]


In [26]:
pd.DataFrame({"korelasi":corr_feature}).to_excel('korelasi variabels.xlsx',index=False)

In [33]:
mean_corr = jum / len(feature_columns)
print('mean: {:.5f}'.format(mean_corr))

# Output: 0.05564

mean: 0.10460


In [35]:
# Menyeleksi setiap fitur yang memiliki nilai korelasi yang kuat
correlations = train_selection[feature_columns + [label_column]].corr()

corr_selection = []

for features in feature_columns:
    correlation_value = correlations.loc[features, label_column]
    
    if correlation_value >= -0.104601 and correlation_value <= 0.104601:
        corr_selection.append(features)
        
print(corr_selection)

# Output: fitur yang akan tereliminasi, diantaranya DC024, DC214, DC217, DC219, DC226, DC232b, DC237a, DC237b, DC237c, DC237d, 
# DC237e, DC237f, DC242, DC252

['DC024', 'DC207', 'DC212', 'DC214', 'DC216', 'DC217', 'DC219', 'DC220', 'DC226', 'DC230b', 'DC232', 'DC232b', 'DC235', 'DC237b', 'DC237c', 'DC237d', 'DC237e', 'DC237f', 'DC241', 'DC242', 'DC252', 'DC142a']


### Drop Values

In [37]:
train_drop = train_selection.copy()

In [38]:
train_drop = train_drop.drop(['id', 'DC024', 'DC207', 'DC212', 'DC214', 'DC216', 'DC217', 'DC219', 'DC220', 'DC226', 'DC230b', 'DC232', 'DC232b', 'DC235', 'DC237b', 'DC237c', 'DC237d', 'DC237e', 'DC237f', 'DC241', 'DC242', 'DC252', 'DC142a'], axis=1)

## Modeling

In [39]:
train_model = train_drop.copy()

In [40]:
train_model.head(10)

,DC025,DC205,DC206,DC208,DC209,DC210,DC211,DC213,DC215,DC230a,DC237,DC237a,DC244,DC246,DC270a,DC109,DC201
0,2,16.0,1.0,1.0,0.0,0.0,0.0,35.0,31.0,1.0,1.0,1.0,0.0,0.0,1,12.0,1
1,2,31.0,1.0,1.0,0.0,0.0,1.0,35.0,31.0,1.0,1.0,1.0,1.0,1.0,2,31.0,1
2,2,12.0,1.0,1.0,1.0,1.0,1.0,35.0,31.0,1.0,1.0,1.0,1.0,1.0,4,11.0,1
3,2,12.0,1.0,0.0,0.0,0.0,1.0,35.0,31.0,1.0,1.0,1.0,1.0,0.0,1,11.0,1
4,2,12.0,1.0,1.0,1.0,0.0,0.0,35.0,31.0,2.0,1.0,1.0,0.0,1.0,2,11.0,1
5,2,16.0,1.0,0.0,0.0,0.0,1.0,11.0,31.0,1.0,1.0,1.0,0.0,0.0,1,12.0,1
6,2,31.0,1.0,1.0,0.0,0.0,1.0,35.0,31.0,1.0,1.0,1.0,1.0,0.0,2,31.0,0
7,2,12.0,1.0,1.0,1.0,1.0,1.0,33.0,31.0,2.0,1.0,1.0,1.0,1.0,5,11.0,1
8,2,12.0,1.0,0.0,0.0,0.0,1.0,21.0,31.0,1.0,1.0,1.0,0.0,1.0,1,11.0,1
9,2,21.0,1.0,1.0,1.0,0.0,1.0,21.0,31.0,1.0,1.0,1.0,1.0,0.0,3,41.0,1


In [41]:
X = train_model.drop('DC201', 1)
y = train_model['DC201']

C:\Users\FADLAN~1\AppData\Local\Temp/ipykernel_16312/2894025009.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = train_model.drop('DC201', 1)


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.26, random_state=200)

In [51]:
X_test.shape

(9353, 16)

In [49]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

from sklearn.metrics import confusion_matrix, precision_score, recall_score
print('Akurasi:',acc)
print('F1 Skor:',f1)
print('\n')
print(precision_score(y_pred=y_pred, y_true=y_test))
print(recall_score(y_pred=y_pred, y_true=y_test))
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
print(tn, fp, fn, tp)
print(classification_report(y_test, y_pred))

Akurasi: 0.9069817170961189
F1 Skor: 0.9500516706854978


0.9224080267558529
0.9794034090909091
209 696 174 8274
              precision    recall  f1-score   support

           0       0.55      0.23      0.32       905
           1       0.92      0.98      0.95      8448

    accuracy                           0.91      9353
   macro avg       0.73      0.61      0.64      9353
weighted avg       0.89      0.91      0.89      9353



## Prediction

In [ ]:
test_df = pd.read_csv('D:/Dataset/water_feasibility/test.csv', sep=',')

In [ ]:
test_df = test_df.drop(['id', 'DC024', 'DC214', 'DC217', 'DC219', 'DC226', 
                              'DC232b', 'DC237a', 'DC237b', 'DC237c', 'DC237d', 'DC237e', 
                              'DC237f', 'DC242', 'DC252'], axis=1)

In [ ]:
y_test_pred = model.predict(test_df)

In [ ]:
y_test_pred

In [ ]:
# Konversi tipe data pada variabel y_test_pred
y_test_pred = y_test_pred.astype('str')

In [ ]:
# Melabeli setiap nilai variabel y_test_pred
y_test_pred[y_test_pred == '0'] = 'Tidak Layak Minum'
y_test_pred[y_test_pred == '1'] = 'Layak Minum'

In [ ]:
submission = pd.read_csv('D:/Dataset/water_feasibility/sample_submission.csv', sep=',')
submission['DC201'] = y_test_pred

In [ ]:
submission.head(50)

In [ ]:
submission.to_csv('sample_submission.csv', index=False)